In [9]:
#Correlation test vanilla on both real and synthetic data.
import scipy
import pandas as pd
import pickle
import numpy as np
from analysis_util import *
import statsmodels.api as sm
filehander = parse_real_results(save_dict=True, recal=False)
filehandler = open('parsed_results/real_results.obj', 'rb') 
real_results = pd.DataFrame.from_dict(pickle.load(filehandler))

In [10]:
#Do VANILLA regression test here (across all models).
#DO I NEED TO REMOVE RANDOM SAMPLING HERE?

aggregated_real = aggregate_real_results(real_results)
selection = aggregated_real[aggregated_real['acquisition'] != "RS"]
xy = selection[['calibration_mse', 'tot_regret_pool']]
xy['intercept'] = 1
model = sm.OLS(xy['tot_regret_pool'], xy[['calibration_mse', 'intercept']])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

                            OLS Regression Results                            
Dep. Variable:        tot_regret_pool   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     768.2
Date:                Sun, 12 Feb 2023   Prob (F-statistic):          9.48e-161
Time:                        13:39:23   Log-Likelihood:                -16515.
No. Observations:                7200   AIC:                         3.303e+04
Df Residuals:                    7198   BIC:                         3.305e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
calibration_mse    19.8225      0.715     

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/2299442080.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy['intercept'] = 1


In [11]:
#Regression test with additional intercept for each of the models.
import statsmodels.api as sm
aggregated_real = aggregate_real_results(real_results)
selection = aggregated_real[aggregated_real['acquisition'] != "RS"]
xy = selection[['calibration_mse', 'tot_regret_pool']]
xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
model = sm.OLS(xy['tot_regret_pool'], xy[['calibration_mse', 'BNN', 'DE', 'GP', 'RF']])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

                            OLS Regression Results                            
Dep. Variable:        tot_regret_pool   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     341.3
Date:                Sun, 12 Feb 2023   Prob (F-statistic):          2.02e-269
Time:                        13:39:46   Log-Likelihood:                -16254.
No. Observations:                7200   AIC:                         3.252e+04
Df Residuals:                    7195   BIC:                         3.255e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
calibration_mse    -7.8206      1.804     

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/529568965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/529568965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_379

In [12]:
import statsmodels.api as sm
selection = aggregated_real[aggregated_real['acquisition'] != "RS"]
xy = selection[['calibration_mse', 'inst_regret_pool']]
xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
for model in set(aggregated_real['surrogate']):
    xy[model] = xy[model]*xy['calibration_mse']
xy['intercept'] = 1
model = sm.OLS(xy['inst_regret_pool'], xy[['intercept', 'BNN', 'DE', 'GP', 'RF']])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

                            OLS Regression Results                            
Dep. Variable:       inst_regret_pool   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     357.0
Date:                Sun, 12 Feb 2023   Prob (F-statistic):          8.00e-281
Time:                        13:40:04   Log-Likelihood:                 19425.
No. Observations:                7200   AIC:                        -3.884e+04
Df Residuals:                    7195   BIC:                        -3.881e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0037      0.000      9.138      0.0

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/2210240259.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/2210240259.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_3

In [13]:
#Calculate ELPD
filehandler = open('parsed_results/real_results.obj', 'rb') 
real_results = pd.DataFrame.from_dict(pickle.load(filehandler))
aggregate_results = aggregate_real_results(real_results)

In [20]:
display(aggregate_results)

,surrogate,acquisition,seed,data,dist_nearest_train,inst_regret_test,inst_regret_pool,tot_regret_test,tot_regret_pool,std_change,calibration_mse,sharpness,x_opt_dist_test,x_opt_dist_pool,elpd
0,BNN,UCB,25,fashionmnist,1.310057,0.009012,0.003380,9.077342,8.508445,1.0,0.088364,0.988819,3.109387,2.056537,-8.358932e+01
1,RF,TS,67,fashionmnist,1.096183,0.008770,0.000000,2.668899,1.783089,1.0,0.006928,-0.515190,1.159039,0.000000,-2.814406e+10
2,DE,RS,93,fashionmnist,1.188979,0.066877,0.054267,7.171189,5.897592,1.0,0.010897,-0.094937,2.934265,2.463365,-3.505381e-01
3,GP,TS,40,fashionmnist,0.933712,0.012433,0.007686,5.323852,4.844387,1.0,0.027271,-0.935378,0.782267,2.628219,-1.102442e+00
4,RF,TS,23,fashionmnist,1.052692,0.009154,0.000000,1.250953,0.326413,1.0,0.012390,-0.411589,2.582823,0.000000,-6.081807e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,BNN,TS,85,svm_wine,0.331033,0.000000,0.000000,0.000000,0.000000,1.0,0.097939,1.578599,1.285102,1.346839,-2.353185e+02
9596,GP,UCB,68,svm_wine,0.370231,0.000000,0.000000,0.000000,0.000000,1.0,0.021145,-0.679767,1.099158,2.091784,-1.961209e+00
9597,GP,TS,77,svm_wine,0.377340,0.000000,0.000000,0.000000,0.000000,1.0,0.021799,-0.604018,1.057315,1.576296,-1.989144e+00
9598,GP,RS,52,svm_wine,0.182310,0.000000,0.000000,0.000000,0.000000,1.0,0.010603,0.740182,1.427660,1.324073,-1.395769e+01


In [15]:
elpd_dict = {}
for surrogate in set(aggregate_results['surrogate']):
    for dataset in set(aggregate_results['data']):
        selection = aggregate_results[(aggregate_results['data'] == dataset) & (aggregate_results['surrogate']==surrogate) & (aggregate_results['acquisition'] == "RS")]
        key_dict = surrogate + "-" + dataset
        elpd_dict[key_dict] = selection['elpd'].mean()
json_object = json.dumps(elpd_dict, indent=4)
with open("./parsed_results/elpd_dict.json", "w") as outfile:
    outfile.write(json_object)

In [31]:
import pprint
pprint.pprint(elpd_dict)

{'BNN-fashionmnist': -109.74285199460559,
 'BNN-fashionmnist_cnn': -122.84772437749265,
 'BNN-mnist': -109.74727505975139,
 'BNN-mnist_cnn': -117.31728027891572,
 'BNN-news': -99.82959432509887,
 'BNN-svm_wine': -440.4377067303117,
 'DE-fashionmnist': -0.4665104047148262,
 'DE-fashionmnist_cnn': -0.800890257822449,
 'DE-mnist': -0.44978179115484174,
 'DE-mnist_cnn': -1.0091267699522581,
 'DE-news': -0.7727202567990459,
 'DE-svm_wine': -1.0169329418434585,
 'GP-fashionmnist': -0.6333471760489402,
 'GP-fashionmnist_cnn': -0.9045186820177409,
 'GP-mnist': -0.634615100607856,
 'GP-mnist_cnn': -0.9444660133904585,
 'GP-news': -0.8116723692255825,
 'GP-svm_wine': -3.0086256256038206,
 'RF-fashionmnist': -898401252749.8406,
 'RF-fashionmnist_cnn': -1994689127942.6768,
 'RF-mnist': -199631563408.39346,
 'RF-mnist_cnn': -14323011354750.307,
 'RF-news': -1712951717254.7043,
 'RF-svm_wine': -49135035957640.52}


In [30]:
#Regression test with additional intercept for each of the models.
import statsmodels.api as sm
aggregated_real = aggregate_real_results(real_results)
selection = aggregated_real[aggregated_real['acquisition'] != "RS"]
selection['surr_problem'] = selection.apply(lambda row: row.surrogate + "-" + row.data, axis=1)
xy = selection[['calibration_mse', 'tot_regret_pool']]
xy[list(set(selection['surr_problem']))] = pd.get_dummies(selection['surr_problem'])[list(set(selection['surr_problem']))]
for index, row in xy.iterrows():
    for combination in set(selection['surr_problem']):
        xy.loc[index,combination] = row[combination]*elpd_dict[combination]
display(xy)
xy[['BNN', 'DE', 'GP', 'RF']] = pd.get_dummies(selection['surrogate'])[['BNN', 'DE', 'GP', 'RF']]
independents = ['BNN', 'DE', 'GP', 'RF']
independents.extend(list(set(selection['surr_problem'])))
model = sm.OLS(xy['tot_regret_pool'], xy[independents])
results=model.fit()
print(results.summary())
#scipy.stats.linregress(aggregated_synth['calibration_mse'], aggregated_synth['inst_regret_pool'])

/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/3734168800.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection['surr_problem'] = selection.apply(lambda row: row.surrogate + "-" + row.data, axis=1)
/var/folders/pg/8kgknkg97zgbj393j1nl2gdh0000gn/T/ipykernel_37904/3734168800.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xy[list(set(selection['surr_problem']))] = pd.get_dummies(selection['surr_problem'])[list(set(selection['surr_problem']))]
/var/folders/pg/8kgknkg97zgbj393j1nl2

,calibration_mse,tot_regret_pool,RF-mnist,GP-fashionmnist_cnn,DE-svm_wine,RF-news,BNN-fashionmnist,GP-mnist_cnn,BNN-mnist_cnn,RF-fashionmnist,...,DE-news,BNN-fashionmnist_cnn,BNN-mnist,BNN-svm_wine,DE-fashionmnist_cnn,RF-fashionmnist_cnn,GP-mnist,GP-news,RF-svm_wine,BNN-news
0,0.088364,8.508445,0.0,0.0,0.000000,0.0,-109.742852,0.0,0.0,0.000000e+00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.006928,1.783089,0.0,0.0,0.000000,0.0,-0.000000,0.0,0.0,-8.984013e+11,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.027271,4.844387,0.0,0.0,0.000000,0.0,-0.000000,0.0,0.0,-0.000000e+00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.012390,0.326413,0.0,0.0,0.000000,0.0,-0.000000,0.0,0.0,-8.984013e+11,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,0.019078,0.978926,0.0,0.0,0.000000,0.0,-0.000000,0.0,0.0,-8.984013e+11,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9594,0.096590,0.087322,-0.0,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000e+00,...,-0.0,-0.0,-0.0,-440.437707,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
9595,0.097939,0.000000,-0.0,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000e+00,...,-0.0,-0.0,-0.0,-440.437707,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
9596,0.021145,0.000000,-0.0,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000e+00,...,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
9597,0.021799,0.000000,-0.0,-0.0,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.000000e+00,...,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


                            OLS Regression Results                            
Dep. Variable:        tot_regret_pool   R-squared:                       0.458
Model:                            OLS   Adj. R-squared:                  0.457
Method:                 Least Squares   F-statistic:                     264.1
Date:                Sun, 12 Feb 2023   Prob (F-statistic):               0.00
Time:                        13:57:18   Log-Likelihood:                -14672.
No. Observations:                7200   AIC:                         2.939e+04
Df Residuals:                    7176   BIC:                         2.956e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
BNN                      0.0017 